In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql.functions import isnan, when, count, concat, lit, col, explode,replace, regexp_replace, udf,length, lower, regexp_replace, array_union, array_join
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, BooleanType,ArrayType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pyspark.sql.utils
import nltk
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect

creamos sesion para usar pyspark


In [2]:
SpSesion = SparkSession \
          .builder \
          .appName('etl') \
          .getOrCreate() 

In [3]:
SpContext = SpSesion.sparkContext

vamos a cargar datos


In [4]:
path = r'C:\Users\Faccu E\Downloads\PI01\FastAPI\Data\user_reviews_modificado.json'

In [5]:
schema = StructType([
        StructField("user_id", StringType()),
        StructField("user_url", StringType()),
        StructField("reviews", ArrayType(StructType([
            StructField("funny", StringType()),
            StructField("posted", StringType()),
            StructField("last_edited", StringType()),
            StructField("item_id", StringType()),
            StructField("helpful", StringType()),
            StructField("recommend", BooleanType()),
            StructField("review", StringType())
    ])))
])

In [6]:
dataset = SpSesion.read.json(path,schema=schema)

Vamos a pasar a explorar el archivo y ver de que se trata, tipos de columnas variables  etc


In [7]:
dataset.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_url: string (nullable = true)
 |-- reviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- funny: string (nullable = true)
 |    |    |-- posted: string (nullable = true)
 |    |    |-- last_edited: string (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- helpful: string (nullable = true)
 |    |    |-- recommend: boolean (nullable = true)
 |    |    |-- review: string (nullable = true)



In [8]:
print('los tipos de datos son para cada columna \n',dataset.dtypes)
print()
print(f'tiene {len(dataset.columns)} columnas ')
print('las columnas son \n',dataset.columns)
print()
print(f'el dataset tiene {dataset.count()} filas')
print()



los tipos de datos son para cada columna 
 [('user_id', 'string'), ('user_url', 'string'), ('reviews', 'array<struct<funny:string,posted:string,last_edited:string,item_id:string,helpful:string,recommend:boolean,review:string>>')]

tiene 3 columnas 
las columnas son 
 ['user_id', 'user_url', 'reviews']

el dataset tiene 25798 filas



eliminamos  la columna de datos corruptos


In [9]:

dataset = dataset.drop("_corrupt_record") 

In [10]:
dataset

DataFrame[user_id: string, user_url: string, reviews: array<struct<funny:string,posted:string,last_edited:string,item_id:string,helpful:string,recommend:boolean,review:string>>]

In [11]:
dataset.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_url: string (nullable = true)
 |-- reviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- funny: string (nullable = true)
 |    |    |-- posted: string (nullable = true)
 |    |    |-- last_edited: string (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- helpful: string (nullable = true)
 |    |    |-- recommend: boolean (nullable = true)
 |    |    |-- review: string (nullable = true)



In [12]:
print('los tipod de tados son  para cada columna \n',dataset.dtypes)
print()
print(f'tiene {len(dataset.columns)} columnas ')
print('las columnas son \n',dataset.columns)
print()
print(f'el dataset tiene {dataset.count()} filas')
print()




los tipod de tados son  para cada columna 
 [('user_id', 'string'), ('user_url', 'string'), ('reviews', 'array<struct<funny:string,posted:string,last_edited:string,item_id:string,helpful:string,recommend:boolean,review:string>>')]

tiene 3 columnas 
las columnas son 
 ['user_id', 'user_url', 'reviews']

el dataset tiene 25798 filas



In [13]:
dataset.show(100)

+--------------------+--------------------+--------------------+
|             user_id|            user_url|             reviews|
+--------------------+--------------------+--------------------+
|   76561197970982479|http://steamcommu...|[{NULL, Posted No...|
|             js41637|http://steamcommu...|[{NULL, Posted Ju...|
|           evcentric|http://steamcommu...|[{NULL, Posted Fe...|
|               doctr|http://steamcommu...|[{NULL, Posted Oc...|
|           maplemage|http://steamcommu...|[{NULL, Posted Ap...|
|              Wackky|http://steamcommu...|[{NULL, Posted Ma...|
|   76561198079601835|http://steamcommu...|[{NULL, Posted Ma...|
|         MeaTCompany|http://steamcommu...|[{NULL, Posted Ju...|
|   76561198089393905|http://steamcommu...|[{NULL, Posted Fe...|
|   76561198156664158|http://steamcommu...|[{NULL, Posted Ju...|
|   76561198077246154|http://steamcommu...|[{NULL, Posted Ju...|
|          WeiEDKrSat|http://steamcommu...|[{NULL, Posted De...|
|       thequeenpanda|htt

In [14]:
df_reviews = dataset.select(explode(col("reviews")).alias("reviews"))
df_reviews.printSchema()


root
 |-- reviews: struct (nullable = true)
 |    |-- funny: string (nullable = true)
 |    |-- posted: string (nullable = true)
 |    |-- last_edited: string (nullable = true)
 |    |-- item_id: string (nullable = true)
 |    |-- helpful: string (nullable = true)
 |    |-- recommend: boolean (nullable = true)
 |    |-- review: string (nullable = true)



Hacemos un Explode de la columna review

In [15]:
df_explotado = dataset.select("user_id", "user_url", explode("reviews").alias("review"))

df_explotado.show(20, truncate=False)

+-----------------+----------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id          |user_url                                            |review                                                                                                                                                            

Buscamos datos nulos en la columna Reviews

In [16]:
df_null_reviews = dataset.filter(dataset.reviews.isNull())

num_null_rows = df_null_reviews.count()

print("El dataset tiene", num_null_rows, "datos nulos en la columna 'reviews'")

El dataset tiene 627 datos nulos en la columna 'reviews'


In [17]:
df = dataset.withColumn("reviews", explode(dataset.reviews))

Asignamos los nombres de las columnas que estamos creando

In [18]:
df = df.withColumn("funny", col("reviews.funny"))
df = df.withColumn("posted", col("reviews.posted"))
df = df.withColumn("last_edited", col("reviews.last_edited"))
df = df.withColumn("item_id", col("reviews.item_id"))
df = df.withColumn("helpful", col("reviews.helpful"))
df = df.withColumn("recommend", col("reviews.recommend"))
df = df.withColumn("review", col("reviews.review"))

Eliminamos la columna reviews

In [19]:
df = df.drop("reviews")

In [20]:
df.show(truncate=False)

+-----------------+----------------------------------------------------+-----+-------------------------+-----------------------------+-------+-----------------------------------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id          |user_url                                            |funny|posted                   |last_edited                  |item_id|helpful                                        |recommend|

Empezamos a limpiar las columnas que no necesitamos

In [21]:
df = df.drop('funny')
df = df.drop('helpful')

In [22]:
df.show(truncate=False)

+-----------------+----------------------------------------------------+-------------------------+-----------------------------+-------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id          |user_url                                            |posted                   |last_edited                  |item_id|recommend|review                                                                                                      

buscamos caracteristicas dentro de la columna review

In [23]:
total_rows = df.count()
unique_apps = df.select("review").distinct().count()
null_apps = df.filter(col("review").isNull()).count()
repeated_apps = total_rows - unique_apps - null_apps

print("Número total de filas:", total_rows)
print("Número de aplicaciones únicas:", unique_apps)
print("Número de aplicaciones con valores nulos:", null_apps)
print("Número de juegos repetidos:", repeated_apps)

Número total de filas: 56642
Número de aplicaciones únicas: 52685
Número de aplicaciones con valores nulos: 0
Número de juegos repetidos: 3957


Ahora haremos trabajando unicamente con el dataframe en pandas  usando la columna review  usamos langdetect para determinar el lenguaje de dicha columna

hacemos la funcion para detectar el idioma

In [24]:
df = df.withColumn("review", lower(col("review")))

In [25]:
df = df.withColumn("review", regexp_replace(col("review"), "[^a-zA-Z\s]", ""))

In [26]:
tokenizer = Tokenizer(inputCol="review", outputCol="words")
df = tokenizer.transform(df)

In [27]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df = remover.transform(df)

In [28]:
df = df.withColumn("normalized_text", F.concat_ws(" ", col("filtered_words")))

Clasificamos los reviews

In [29]:
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "buena"
    elif polarity < -0.1:
        return "mala"
    else:
        return "neutral"

In [30]:
get_sentiment_udf = F.udf(get_sentiment)

In [31]:
df = df.withColumn("sentiment", get_sentiment_udf(df["normalized_text"]))

In [32]:
df.show()

+-----------------+--------------------+--------------------+--------------------+-------+---------+--------------------+--------------------+--------------------+--------------------+---------+
|          user_id|            user_url|              posted|         last_edited|item_id|recommend|              review|               words|      filtered_words|     normalized_text|sentiment|
+-----------------+--------------------+--------------------+--------------------+-------+---------+--------------------+--------------------+--------------------+--------------------+---------+
|76561197970982479|http://steamcommu...|Posted November 5...|                    |   1250|     true|simple yet with g...|[simple, yet, wit...|[simple, yet, gre...|simple yet great ...|    buena|
|76561197970982479|http://steamcommu...|Posted July 15, 2...|                    |  22200|     true|its unique and wo...|[its, unique, and...|[unique, worth, p...|unique worth play...|    buena|
|76561197970982479|http:/

Eliminamos las columnas que no necesitamos

In [33]:
df = df.drop('review')
df = df.drop('words')
df = df.drop('filtered_words')
df = df.drop('normalized_text')
df = df.drop('last_edited')

Eliminamos la palabra "Posted" de la columna posted

In [34]:
df = df.withColumn("posted", regexp_replace("posted", "Posted", ""))

agregamos un espacio al final de cada fila en la columna user_url para que no toque con la columna siguiente y arroje error el link

In [35]:
df = df.withColumn("user_url", concat(col("user_url"), lit(" ")))

In [36]:
df.show(truncate=False)

+-----------------+-----------------------------------------------------+-------------------+-------+---------+---------+
|user_id          |user_url                                             |posted             |item_id|recommend|sentiment|
+-----------------+-----------------------------------------------------+-------------------+-------+---------+---------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 | November 5, 2011. |1250   |true     |buena    |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 | July 15, 2011.    |22200  |true     |buena    |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 | April 21, 2011.   |43110  |true     |neutral  |
|js41637          |http://steamcommunity.com/id/js41637                 | June 24, 2014.    |251610 |true     |buena    |
|js41637          |http://steamcommunity.com/id/js41637                 | September 8, 2013.|227300 |true     |neutral  |
|js41637          |http:

Eliminamos la columna user_url dado que no la vamos a utilizar

In [37]:
df = df.drop('user_url')

transformamos el df a Pandas

In [38]:
df_pandas = df.toPandas()

Corroboramos que se haya transformado correctamente

In [39]:
df_pandas

,user_id,posted,item_id,recommend,sentiment
0,76561197970982479,"November 5, 2011.",1250,True,buena
1,76561197970982479,"July 15, 2011.",22200,True,buena
2,76561197970982479,"April 21, 2011.",43110,True,neutral
3,js41637,"June 24, 2014.",251610,True,buena
4,js41637,"September 8, 2013.",227300,True,neutral
...,...,...,...,...,...
56637,76561198312638244,July 10.,70,True,buena
56638,76561198312638244,July 8.,362890,True,buena
56639,LydiaMorley,July 3.,273110,True,neutral
56640,LydiaMorley,July 20.,730,True,neutral


voy a separar los años de posteo

In [40]:
df_pandas['posted'] = df_pandas['posted'].replace(r'^\s*\w+\s*\d+\.$', np.nan, regex=True)

df_pandas['posted'] = pd.to_datetime(df_pandas['posted'], errors='coerce')

df_pandas['year'] = df_pandas['posted'].dt.year

In [41]:
df_pandas

,user_id,posted,item_id,recommend,sentiment,year
0,76561197970982479,2011-11-05,1250,True,buena,2011.0
1,76561197970982479,2011-07-15,22200,True,buena,2011.0
2,76561197970982479,2011-04-21,43110,True,neutral,2011.0
3,js41637,2014-06-24,251610,True,buena,2014.0
4,js41637,2013-09-08,227300,True,neutral,2013.0
...,...,...,...,...,...,...
56637,76561198312638244,NaT,70,True,buena,NaN
56638,76561198312638244,NaT,362890,True,buena,NaN
56639,LydiaMorley,NaT,273110,True,neutral,NaN
56640,LydiaMorley,NaT,730,True,neutral,NaN


voy a eliminar las filas que no tienen año de posteo

In [42]:
num_valores_null = df_pandas['year'].isnull().sum()

print(num_valores_null)


9588


In [43]:
df_pandas = df_pandas.dropna(subset=['year'])

In [44]:
df_pandas

,user_id,posted,item_id,recommend,sentiment,year
0,76561197970982479,2011-11-05,1250,True,buena,2011.0
1,76561197970982479,2011-07-15,22200,True,buena,2011.0
2,76561197970982479,2011-04-21,43110,True,neutral,2011.0
3,js41637,2014-06-24,251610,True,buena,2014.0
4,js41637,2013-09-08,227300,True,neutral,2013.0
...,...,...,...,...,...,...
56589,wayfeng,2015-10-14,730,True,neutral,2015.0
56592,76561198251004808,2015-10-10,253980,True,buena,2015.0
56602,72947282842,2015-10-31,730,True,mala,2015.0
56604,ApxLGhost,2015-12-14,730,True,buena,2015.0


In [45]:
df_pandas['year'] = df_pandas['year'].astype(int)

C:\Users\Faccu E\AppData\Local\Temp\ipykernel_8228\168905133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas['year'] = df_pandas['year'].astype(int)


In [46]:
df_pandas

,user_id,posted,item_id,recommend,sentiment,year
0,76561197970982479,2011-11-05,1250,True,buena,2011
1,76561197970982479,2011-07-15,22200,True,buena,2011
2,76561197970982479,2011-04-21,43110,True,neutral,2011
3,js41637,2014-06-24,251610,True,buena,2014
4,js41637,2013-09-08,227300,True,neutral,2013
...,...,...,...,...,...,...
56589,wayfeng,2015-10-14,730,True,neutral,2015
56592,76561198251004808,2015-10-10,253980,True,buena,2015
56602,72947282842,2015-10-31,730,True,mala,2015
56604,ApxLGhost,2015-12-14,730,True,buena,2015


Creamos un CSV final con el ETL Realizado

In [47]:
df_pandas.to_parquet('user_reviews_final.parquet', index=False)